In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import glob
import zipfile
import random
import shutil

root_ = '/content'
drive_dir = os.path.join(root_,'drive','MyDrive')
train_ = '/dataset/train'
os.chdir(os.path.join(root_))

In [ ]:
zip_ref = zipfile.ZipFile("/content/drive/My Drive/dog-breed-identification.zip", 'r')
zip_ref.extractall("/content/dataset/")
zip_ref.close()

In [ ]:
os.makedirs('/content/dataset/valid')

In [ ]:
import pandas as pd
labels_csv = pd.read_csv('/content/dataset/labels.csv')

In [ ]:
os.chdir('/content/dataset/train/')
  
for c in random.sample(glob.glob('*.jpg'),2000):
  shutil.move(c,os.path.join('/content/dataset/valid/'))

In [ ]:
for breed in labels_csv.breed.unique():
  os.makedirs(os.path.join('/content/dataset/train',breed))
  os.makedirs(os.path.join('/content/dataset/valid',breed))

In [ ]:
train_path = '/content/dataset/train'#os.path.join(root_,'train')
test_path = '/content/dataset/test'#os.path.join(root_,'test')
#valid_path = '/content/dataset/valid'#os.path.join(root_,'valid')

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

In [ ]:
os.chdir('/content/dataset/train/')

for id,breed in zip(labels_csv.id,labels_csv.breed):
  
  try:
    shutil.move('/content/dataset/train/'+id+'.jpg','/content/dataset/train/'+breed)
  except:
    shutil.move('/content/dataset/valid/'+id+'.jpg','/content/dataset/valid/'+breed)

# **Steps** <p>
Step 1: Load Dataset <p>
Step 2: Transform the Dataset <p>
Step 3: Create Model <p>
Step 4: Train Model <p>
Step 5: Save the Model <p>
Step 6: Load the Model <p>
Step 7: Predict the Image <p>
Step 8: Show the result

# **Step 1: Load Dataset**

In [ ]:
#cp -av /content/drive/MyDrive/Dataset/ImageClassification-20211203T192447Z-001.zip .

In [ ]:
#!unzip  "ImageClassification-20211203T192447Z-001.zip"

In [ ]:
#cp -av /content/drive/MyDrive/Dataset/Dataset.zip .

In [ ]:
#!unzip  "Dataset.zip"

# **Step 2: Importing Important Libraries**

In [ ]:
#Import lib
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from keras.optimizers import gradient_descent_v2
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
import tensorflow as tf
from keras.callbacks import TensorBoard as tensorboard
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K
import matplotlib.pyplot as plt
# from pyimagesearch.smallervggnet import SmallerVGGNet
from imutils import paths
import numpy as np
import argparse
import random
import pickle
import cv2
import os

In [ ]:
# initialize the number of epochs to train for, initial learning rate,
# batch size, and image dimensions
EPOCHS = 10
INIT_LR = 1e-3
BS = 5
IMAGE_DIMS = (96, 96, 3)


In [ ]:
imagePaths = sorted(list(paths.list_images("/content/dataset/train/")))
random.seed(42)
random.shuffle(imagePaths)

In [ ]:
data = []
labels = []

# loop over the input images
for imagePath in imagePaths:
	# load the image, pre-process it, and store it in the data list
	image = cv2.imread(imagePath)
	image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
	image = img_to_array(image)
	data.append(image)

	# extract set of class labels from the image path and update the
	# labels list
	l = label = imagePath.split(os.path.sep)[-2].split("_")
	labels.append(l)

In [ ]:
del image

In [ ]:
#labels = ['affenpinscher','afghan_hound','african_hunting_dog','airedale','american_staffordshire_terrier','appenzeller','australian_terrier','basenji','basset','beagle','bedlington_terrier','bernese_mountain_dog','black-and-tan_coonhound','blenheim_spaniel','bloodhound','bluetick','border_collie','border_terrier','borzoi','boston_bull','bouvier_des_flandres','boxer','brabancon_griffon','briard','brittany_spaniel','bull_mastiff','cairn','cardigan','chesapeake_bay_retriever','chihuahua','chow','clumber','cocker_spaniel','collie','curly-coated_retriever','dandie_dinmont','dhole','dingo','doberman','english_foxhound','english_setter','english_springer','entlebucher','eskimo_dog','flat-coated_retriever','french_bulldog','german_shepherd','german_short-haired_pointer','giant_schnauzer','golden_retriever','gordon_setter','great_dane','great_pyrenees','greater_swiss_mountain_dog','groenendael','ibizan_hound','irish_setter','irish_terrier','irish_water_spaniel','irish_wolfhound','italian_greyhound','japanese_spaniel','keeshond','kelpie','kerry_blue_terrier','komondor','kuvasz','labrador_retriever','lakeland_terrier','leonberg','lhasa','malamute','malinois','maltese_dog','mexican_hairless','miniature_pinscher','miniature_poodle','miniature_schnauzer','newfoundland','norfolk_terrier','norwegian_elkhound','norwich_terrier','old_english_sheepdog','otterhound','papillon','pekinese','pembroke','omeranian','pug','redbone','rhodesian_ridgeback','rottweiler','saint_bernard','saluki','samoyed','schipperke','scotch_terrier','scottish_deerhound','sealyham_terrier','shetland_sheepdog','shih-tzu','siberian_husky','silky_terrier','soft-coated_wheaten_terrier','staffordshire_bullterrier','standard_poodle','standard_schnauzer','sussex_spaniel','tibetan_mastiff','tibetan_terrier','toy_poodle','toy_terrier','vizsla','walker_hound','weimaraner','welsh_springer_spaniel','west_highland_white_terrier','whippet','wire-haired_fox_terrier','yorkshire_terrier']

### To check the total number of images present in the dataset

In [ ]:
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)
print("[INFO] data matrix: {} images ({:.2f}MB)".format(
	len(imagePaths), data.nbytes / (1024 * 1000.0)))

# binarize the labels using scikit-learn's special multi-label
# binarizer implementation

mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(labels)
print("[INFO] class labels:")

# **Checking the Labels**

In [ ]:
# loop over each of the possible class labels and show them
for (i, label) in enumerate(mlb.classes_):
	print("{}. {}".format(i + 1, label))


# **Train-Test Split**

In [ ]:
# the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.2, random_state=42)

In [ ]:
del data

In [ ]:
del labels

# **Data Augmentation**

### Generated a data augmentation inorder to increase the amount of dataset and avoid overfitting

In [ ]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rescale=1./255,rotation_range=25, width_shift_range=0.1,
	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	horizontal_flip=True, fill_mode="nearest")

# **Creating a OneLayerNetwork Model**

### Added activation function as SELU

In [ ]:
  def onelayernetwork(width, height, depth, classes, finalAct):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1

		# if we are using "channels first", update the input shape
		# and channels dimension
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1

		# CONV => RELU => POOL
		model.add(Conv2D(32, (3, 3), padding="same",
			input_shape=inputShape))
		model.add(Activation("selu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(3, 3)))
		model.add(Dropout(0.3))

		# # (CONV => RELU) * 2 => POOL
		#model.add(Conv2D(64, (3, 3), padding="same"))
		#model.add(Activation("selu"))
		#model.add(BatchNormalization(axis=chanDim))
		#model.add(Conv2D(64, (3, 3), padding="same"))
		#model.add(Activation("selu"))
		#model.add(BatchNormalization(axis=chanDim))
		#model.add(MaxPooling2D(pool_size=(2, 2)))
		#model.add(Dropout(0.25))

		# # (CONV => RELU) * 2 => POOL
		# model.add(Conv2D(128, (3, 3), padding="same"))
		# model.add(Activation("selu"))
		# model.add(BatchNormalization(axis=chanDim))
		# model.add(Conv2D(128, (3, 3), padding="same"))
		# model.add(Activation("selu"))
		# model.add(BatchNormalization(axis=chanDim))
		# model.add(MaxPooling2D(pool_size=(2, 2)))
		# model.add(Dropout(0.25))

		# first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(1024))
		model.add(Activation("selu"))
		model.add(BatchNormalization())
		model.add(Dropout(0.5))

		# softmax classifier
		model.add(Dense(classes))
		model.add(Activation(finalAct))

		# return the constructed network architecture
		return model

In [ ]:
model1 = onelayernetwork(
	width=IMAGE_DIMS[1], height=IMAGE_DIMS[0],
	depth=IMAGE_DIMS[2], classes=len(mlb.classes_),
	finalAct="sigmoid")


# **Gradient Estimation**

In [ ]:
# initialize the optimizer (SGD is sufficient)
opt = tf.keras.optimizers.SGD(learning_rate=0.1)

## Tensorboard

In [ ]:
import datetime

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


# **Cost Function**

In [ ]:
# compile the model using binary cross-entropy rather than
# categorical cross-entropy -- this may seem counterintuitive for
# multi-label classification, but keep in mind that the goal here
# is to treat each output label as an independent Bernoulli
# distribution
model1.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

In [ ]:
# Tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
H = model1.fit_generator(
	aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY),
	steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS, verbose=1,callbacks=[tensorboard_callback])

# Model 


In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
from tensorboard import notebook
notebook.list()

In [ ]:
print("[INFO] serializing network...")
model1.save("Car1.model")

In [ ]:
model1.save_weights("model1.h5")
print("Saved model to disk")

In [ ]:
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_accuracy")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper left")
# plt.savefig(args["plot"])

In [ ]:
f = open("mlb.pickle", "wb")
f.write(pickle.dumps(mlb))
f.close()


Test the model

In [ ]:
pip install imutils 


In [ ]:
import imutils 

In [ ]:
image = cv2.imread("/content/Dataset/Damaged Images/113198_PhotoGrid_1474238007772.jpg")
output = imutils.resize(image, width=600)
 
# pre-process the image for classification
image = cv2.resize(image, (96, 96))
image = image.astype("float") / 255.0
image = img_to_array(image)
image = np.expand_dims(image, axis=0)

In [ ]:
mlb = pickle.loads(open("mlb.pickle", "rb").read())


In [ ]:
print("[INFO] classifying image...")
proba = model1.predict(image)[0]
idxs = np.argsort(proba)[::-1][:]

In [ ]:
proba

In [ ]:
label2={}

# loop over the indexes of the high confidence class labels
for (i, j) in enumerate(idxs):
	# build the label and draw the label on the image
    #if (mlb.classes_["Front-smash"])
	#label = "{}: {:.2f}%".format(mlb.classes_[j], proba[j] * 100)
    
	label2.update({mlb.classes_[j]:proba[j] * 100})

print(type(mlb.classes_))  

In [ ]:
label2

# **Creating a TwoLayerNetwork Model**

### Added activation function as tanh

In [ ]:
# initialize the number of epochs to train for, initial learning rate,
# batch size, and image dimensions
EPOCHS = 10
INIT_LR = 1e-3
BS = 32
IMAGE_DIMS = (96, 96, 3)

In [ ]:
  def twolayernetwork(width, height, depth, classes, finalAct):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1

		# if we are using "channels first", update the input shape
		# and channels dimension
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1

		# CONV => RELU => POOL
		model.add(Conv2D(32, (3, 3), padding="same",
			input_shape=inputShape))
		model.add(Activation("tanh"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(3, 3)))
		model.add(Dropout(0.25))

		# (CONV => RELU) * 2 => POOL
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("tanh"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("tanh"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))

		# (CONV => RELU) * 2 => POOL
		model.add(Conv2D(128, (3, 3), padding="same"))
		model.add(Activation("selu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(128, (3, 3), padding="same"))
		model.add(Activation("selu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))

		# first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(1024))
		model.add(Activation("tanh"))
		model.add(BatchNormalization())
		model.add(Dropout(0.5))

		# sigmoid classifier
		model.add(Dense(classes))
		model.add(Activation(finalAct))

		# return the constructed network architecture
		return model

In [ ]:
model = twolayernetwork(
	width=IMAGE_DIMS[1], height=IMAGE_DIMS[0],
	depth=IMAGE_DIMS[2], classes=len(mlb.classes_),
	finalAct="sigmoid")

In [ ]:
# initialize the optimizer (SGD is sufficient)
opt = tf.keras.optimizers.RMSprop(learning_rate=0.1)

## Tensorboard

In [ ]:
import datetime

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


# **Cost Function**

In [ ]:
# compile the model using binary cross-entropy rather than
# categorical cross-entropy -- this may seem counterintuitive for
# multi-label classification, but keep in mind that the goal here
# is to treat each output label as an independent Bernoulli
# distribution
model.compile(loss="sparse_categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

In [ ]:
# Tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
H = model.fit_generator(
	aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY),
	steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS, verbose=1)